(Sim_Accel)=
# Improving Sampling and Accelerating Simulations

In many cases of interest, carrying out robust free energy calculations may require significant investment of computational resources, beyond that which can be obtained by most researchers. This page examines additional tools for accelerating the sampling. As these begin to branch out from the [fundamentals of free energy](fundamentals), we will not go deeply into all these methods. They are not needed to carry out free energy calculations, but they may be extremely useful to converge calculations in complex systems with slow dynamics and we encourage readers to examine these topics to see what will work for them.

## Umbrella Sampling
One standard method for improving sampling in atomistic simulations is umbrella sampling,{cite}`Torrie1977` where bias terms are added to constrain the simulation in some way, then the restraints are removed. This method can lower barriers on the potential energy surface, or restrain simulations to slow-interconverting configurations that are relevant to the binding affinities (e.g. different torsional states). Doing this allows for the free energy components to be properly computed and then combined.{cite}`Wang2006,Mobley2007a,Mobley2007b`
If a system has slow degrees of freedom, like some hydration free energies,{cite}`Klimovich2010` this method allows one to sample more frequently in the slow state.

One can also envision using umbrella sampling to compute the free energy of constraining the free ligand into the bound conformation directly before computing the free energy of binding, and then computing the free energy of releasing these restraints in solution. Doing so usually decreases correlation times for sampling intermediate states, thereby increasing the simulation efficiency. {cite}`Wang2006,Woo2005`

## Hamiltonian Replica Exchange
Although it is perfectly valid to run each intermediate state as its own simulation, there are tools that let you run parallel, coupled simulations and swap information between them to improve sampling. One such method is called Hamiltonian Exchange (HEX) or Hamiltonian Replica Exchange (HREX). In HEX, the parallel simulations at each alchemical variable are allowed to swap atoms/molecules (under certain conditions) from state A which has different energy barriers than state B. This enables sampling configurations that may take significant amounts of time to access normally when simulating only one state.
{cite}`Okamoto2004,Faraldo-Gomez2007,Woods2003,Banba2000,Bitetti-Putzer2003,Hritz2008`

We **highly recommend doing HEX methods** as most codes do this on top of well-tested temperature replica exchange methods, and so there is very little overhead. the outputs of Hamiltonian exchange simulations can be analyzed in the same way as the outputs of $K$ uncoupled simulations. These simulations are guaranteed to decorrelate as fast or faster than standard simulations, though the exact amount of improvement depends on the system; at least they will not be any worse!

[Correlation](correlation) times with HEX is a bit more complicated since atoms/molecules are swapping along very different trajectories. In this case, you will need to compute along trajectories that sample different states as opposed to a single state.

### Some Suggestions for Replica Exchange Simulations

* The level of acceleration one gets is a function both of phase space mixing between the replicas and the acceleration provided by of replicas kinetics that have faster sampling than the original distribution in the degree of freedom of interest. 
* Choice of Hamiltonian/temperature states depends quite a bit on what one actually wants to do: is it just accelerating kinetics, or does one actually care about properties as a function of temperature or calculating free energies? That can change the choice.  Identifying a choice of replica that accelerates kinetics in desired way is nontrivial problem. 
* Space replicas to be even in phase space overlap.  The easiest way to do this is to guess a spacing, then look at the results pf MBAR /TI after a short run. You can then respace the replicas so the error in the free energies between consecutive states is close to being equal. It is most important to get rid of very bad spacing; there is not much efficiency gain in going from decent spacing to optimal spacing {cite}`Naden2014`. 
* Acceptance rates should be around 38% to maximize mixing in state space, but anywhere in the 25-50% range is nearly as good, so it is not that useful to tune it too much {cite}`Predescu2005`.
    * More precise quantitative measurements of state space mixing are determined by taking the transition matrix between states, and maximizing $1-\lambda_2$, where $\lambda_2$ is the second largest eigenvalue of the transition matrix. {cite}`Chodera2011`  This quantity is known as the spectral gap, and is related to the rate of mixing in state space.  Remember that mixing in state space is insufficient to accelerate sampling; there must be some state where configurations are sampled more quickly. 
* To improve mixing between states, perform multiple replica switches each time one pauses to exchange. {cite}`Chodera2011`  Anecdotal evidence is that effectiveness of multiple switches decreases past 10-100 exchanges, but this is system dependent.
* Exchange should be as frequent as is permissible given the networking and code {cite}`Chodera2011`, as long as the velocities are not reassigned on switches.   Reassigning velocities too frequently slows kinetics and negates the advantages gained by performing rapid replica switches.{cite}`Basconi2013`

## Other Methods
The other methods that will be mentioned briefly are *Expanded Ensemble* and *$\lambda$-dynamics.* Expanded ensemble works by running a single simulation and sampling both the intermediate states and separate coordinates simultaneously. $\lambda$-dynamics works by treating the alchemical variable as dynamical, introducing a fictitious mass corresponding to the $\lambda$ degree of freedom; although this seems rather novel, it is essentially identical to Monte Carlo techniques.{cite}`Banba2000,Guo1998,Kong1996,Li2007a`
$\lambda$-dynamics is showing promise, but is still in preliminary stages of development{cite}`Zheng2008a,Zheng2008b,Min2008,Li2007b,Min2007`

In general, we **do NOT recommend Expanded Ensemble or $\lambda$-dynamics to beginners.** The methods and implementations are not up to the same robustness as HEX yet and there are tweaks and extra parameters that have to be coded to obtain proper convergence. Given more time and development, these methods may become more accessible to beginners and we will be able to recommend them in the future.

### Slow Growth 

In slow-growth methods, the coupling parameter, $\lambda$, is slowly varied over the course of one or several simulations to take a system gradually from $\lambda=0$ to $\lambda=1$; from this, the free energy difference between endpoints is estimated. In equilibrium methods, on the other hand, separate simulations are run at multiple different lambda values and information from the individual simulations is used to estimate free energy differences between adjoining lambda values.

Slow-growth methods have a number of well-documented problems, such as Hamiltonian lag and hysteresis (*add references*). Additionally, if a slow-growth calculation does not lead to a sufficiently precise free energy estimate, the only way to improve the free energy estimate is to repeat the calculation with a slower rate of change in lambda – the simulation cannot be extended to gain additional precision, meaning that significant data can be wasted. Fast-growth methods, while conceptually appealing, do not appear to offer substantial advantages over equilibrium methods.{cite}`Jarzynski2006,Oostenbrink2006`

### Nonequilibrium (Fast Growth) methods 

Fast-growth methods are based on a theorem by Jarzynski in 1997{cite}`Jarzynski1997` that the free energy difference associated with a particular equilibrium process can be computed by taking an the exponential average of the irreversible work done in performing many (potentially rapid) non-equilibrium trials of the same process. 

$ \Delta G = -k_B T \ln \left \langle e^{-W/k_b T} \right \rangle $

Where $ W$ is the work to execute the transition between the initial and final thermodynamic state. When applied to alchemical free energy calculations, this simply amounts to estimating free energy differences by performing many different rapid versions of a slow-growth trial – that is, rapidly changing lambda between two values (i.e. 0 and 1)  in many separate trials, and monitoring the work done each time.  Equilibrium free energy calculations can be thought of as "instantaneous growth" as they rely on estimating the free energy difference between neighboring $\lambda$ values based only on instantaneous evaluations of potential energy differences between $\lambda$ values (or by evaluation of and extrapolation of $dV/d\lambda$ at a particular $\lambda$ value).